# get Tpref & O2thresh for each species
# need to have enough memory (some fields are big), 50GB worked
# save as netcdf

In [ ]:
#get_ipython().system(u'jupyter nbconvert --to=python plot_AGI_toothfish_testing.ipynb')

In [1]:

import xarray as xr
import numpy as np
import sys, os
from pathlib import Path
import glob
from netCDF4 import Dataset


In [2]:
#-----
# Collection of notes:
#
# excluded two species for now (see Anne's mail): 
# 1) Antimora rostrata: don't have the habitat file (habitat outside of SO) -> have it now, just not yet included
# 2) Macrourus whitsoni (only have Tpref, but not o2thresh & AGIcrit) -> in the making
#
# For some species, the variable in AGI_crit file is actually called "o2thresh"
#  I think the values are correct (at least the order of magnitude suggests that they are indeed AGI values), 
#  but to be double-checked
#  The follwoing files/species are affected: 
#      AGIcrit_Chaenodraco_wilsoni.nc
#      AGIcrit_Lepidonotothen_squamifrons.nc
#      AGIcrit_Notothenia_rossii.nc
#      AGIcrit_Chionobathyscus_dewitti.nc
#      AGIcrit_Muraenolepis_microps.nc
#
# Be careful: 
#  the order in which files/species are loaded e.g. for the array species_names does NOT
#   match the order in which the species are in the lists of LWa, LWb, Linf, depth_min & depth_max
#  As a result, I have added a line to make sure the correct index_depth_min/max are stored for each species
#
# python indexing: 
#  when selceting the indices index_depth_min:index_depth_max from an array, one actually does NOT include
#   the index index_depth_max
#  I assume that we want to include that max. depth though, so I added a "+1" (effectively increasing the habitat)
#  -> that impacts the calculation of o2thresh and Tpref, wouldn't it? -> check with Anne3 
# Note after meeting with Anne: we agreed to rather be inclusive (= keep "+1") to 
#    a) add more data and 
#    b) assuming that there is some uncertainty associated with the observation-based max. depth
#
# I had trouble making the volume calculation work when starting from the example script
#    (lots of array dimension mismatches)
#  I re-coded things a bit to make sure all the arrays are the same size when mutiplied
#  I also explicitly kicked out any bathymetry nodes or nodes outside of the habitat 
#
# AA Toothfish: it is still marked that the depth range is not correct (currently assumed to be 0-10m here, 
#   info is misisng in the table) -> check with Jilda
#  -> that impacts the calculation of o2thresh and Tpref, wouldn't it? -> check with Anne
#
# As a result of all these small things, I get different AGU_crit than those stored in the nc files...
#   to be double-checked!!
#
# should it be (W[i]**(1-d))/(Winf[i]**(1-d))    or     (W[i]**(1-d))/(Winf[i])  ?
#    the PDF suggests the latter, checking Clarke2021 suggests the former (I now used the former)
#
#  Tpref & in-situ temp in the equation should be in Kelvin!! (otherwise I get lots of "infinity")
#
#-------

In [3]:
#-----
# based on python code provided by Anne
#-----

basepath="/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/AGI_toothfish_project/" 
no_species=18 # exclude Antimora_rostrata 

# Init
species_names         = [None] * no_species # empty list
species_names_        = [None] * no_species # empty list
habitat_file_names    = [None] * no_species # empty list

habitat_files = Path(basepath + "updated_habitat_files/").glob('*_boolean.nc')  # updated habitat files, Nov 2022!!!!!!
#habitat_files = Path(basepath + "share/").glob('*_boolean.nc') 
# Note: I changed name of file of Antimora_rostrata to *boolean2.nc, so that it won't be included here
for ifile,file in enumerate(habitat_files):   
    habitat_file_names[ifile]    = str(file)
    # Get the species name from the full pathname with a space between
    species_names[ifile]  = '_'.join(os.path.basename(file).split('_')[:-2]).replace('_',' ') 
    if not species_names[ifile] in ['Galiteuthis glacialis','Mesonychoteuthis hamiltoni','Kondakovia longimana']:
        species_names[ifile] = species_names[ifile][8:] # get rid of "Default" or "Reviewed"
    if species_names[ifile][0].isspace():  # get rid of white space if there is any
        species_names[ifile] = species_names[ifile][1:]
    if species_names[ifile] in ['Chionobathyscus dewitti All Suitable Habitat']:
        species_names[ifile] = species_names[ifile][0:23]
    # Get the species name from the full pathname with _ in it
    species_names_[ifile] = species_names[ifile].replace(' ','_')  #'_'.join(os.path.basename(file).split('_')[:-2]) 

#print(habitat_file_names)
print(species_names)
print(len(species_names),'species')
print(species_names_)


['Galiteuthis glacialis', 'Mesonychoteuthis hamiltoni', 'Notothenia rossii', 'Pleuragramma antarctica', 'Trematomus hansoni', 'Chaenodraco wilsoni', 'Cryodraco antarcticus', 'Kondakovia longimana', 'Trematomus eulepidotus', 'Macrourus whitsoni', 'Notothenia coriiceps', 'Neopagetopsis ionah', 'Lepidonotothen squamifrons', 'Chaenocephalus aceratus', 'Chionobathyscus dewitti', 'Muraenolepis microps', 'Bathyraja maccaini', 'Dissostichus mawsoni']
18 species
['Galiteuthis_glacialis', 'Mesonychoteuthis_hamiltoni', 'Notothenia_rossii', 'Pleuragramma_antarctica', 'Trematomus_hansoni', 'Chaenodraco_wilsoni', 'Cryodraco_antarcticus', 'Kondakovia_longimana', 'Trematomus_eulepidotus', 'Macrourus_whitsoni', 'Notothenia_coriiceps', 'Neopagetopsis_ionah', 'Lepidonotothen_squamifrons', 'Chaenocephalus_aceratus', 'Chionobathyscus_dewitti', 'Muraenolepis_microps', 'Bathyraja_maccaini', 'Dissostichus_mawsoni']


In [4]:
#----
# Species information needed for calculation AGI
#----

# define order of species as contained in depth_min, LWa etc.
species_list = ['Cryodraco antarcticus','Neopagetopsis ionah','Trematomus eulepidotus',\
               'Trematomus hansoni','Bathyraja maccaini','Chaenocephalus aceratus',\
               'Notothenia coriiceps','Pleuragramma antarctica','Dissostichus mawsoni',\
                'Macrourus whitsoni',\
               'Lepidonotothen squamifrons','Notothenia rossii','Muraenolepis microps',\
               'Chaenodraco wilsoni','Chionobathyscus dewitti',\
               'Galiteuthis glacialis','Mesonychoteuthis hamiltoni','Kondakovia longimana']
# excluded the following species for now: 
# 1) Antimora rostrata: don't have the habitat file (habitat outside of SO)

# Minimum depth of occurence (meter)
depth_min     = [None] * len(species_names) # Initialize
depth_min[0]  = 90   # 'Cryodraco antarcticus'
depth_min[1]  = 20   #'Neopagetopsis ionah'
depth_min[2]  = 0    # 'Trematomus eulepidotus'
depth_min[3]  = 6    # 'Trematomus hansoni'
depth_min[4]  = 167  # 'Bathyraja maccaini'
depth_min[5]  = 0    # 'Chaenocephalus aceratus'
depth_min[6]  = 0    # 'Notothenia coriiceps'
depth_min[7]  = 0    # 'Pleuragramma antarctica'
depth_min[8]  = 0    # 'Dissostichus mawsoni' ## Assume a wide range for toothfish (max. overlap with prey) ##
depth_min[9]  = 400  # 'Macrourus whitsoni'
depth_min[10] = 10   # 'Lepidonotothen squamifrons'
#depth_min[11] = 350  # 'Antimora rostrata'
depth_min[11] = 0    # 'Notothenia rossii'
depth_min[12] = 10   # 'Muraenolepis microps'
depth_min[13] = 250  # 'Chaenodraco wilsoni'
depth_min[14] = 500  # 'Chionobathyscus dewitti'
depth_min[15] = 200   #'Galiteuthis glacialis'
depth_min[16] = 200   # 'Mesonychoteuthis hamiltoni'
depth_min[17] = 500   # 'Kondakovia longimana'

# Maximum depth of occurence (meter)
depth_max     = [None] * len(species_names) # Initialize
depth_max[0]  = 600  # 'Cryodraco antarcticus'
depth_max[1]  = 900  #'Neopagetopsis ionah'
depth_max[2]  = 700  # 'Trematomus eulepidotus'
depth_max[3]  = 549  # 'Trematomus hansoni'
depth_max[4]  = 500  # 'Bathyraja maccaini'
depth_max[5]  = 770  # 'Chaenocephalus aceratus'
depth_max[6]  = 550  # 'Notothenia coriiceps'
depth_max[7]  = 728  # 'Pleuragramma antarctica'
depth_max[8]  = 2210   # 'Dissostichus mawsoni' ## Assume a wide range for toothfish (max. overlap with prey) ##
depth_max[9]  = 3185 # 'Macrourus whitsoni'
depth_max[10] = 900  # 'Lepidonotothen squamifrons'
#depth_max[11] = 3000 # 'Antimora rostrata'
depth_max[11] = 1000 # 'Notothenia rossii'
depth_max[12] = 1600 # 'Muraenolepis microps'
depth_max[13] = 800  # 'Chaenodraco wilsoni'
depth_max[14] = 2000 # 'Chionobathyscus dewitti'
depth_max[15] = 2500   #'Galiteuthis glacialis'
depth_max[16] =  600  # 'Mesonychoteuthis hamiltoni'
depth_max[17] = 2000   # 'Kondakovia longimana'

#---
# NOTE Jan 2023: this info below is not needed anymore with the new AGI formulation!
#----
# LWa extracted 21.07.2022 from FishBase
LWa     = [None] * len(species_names) # Initialize
LWa[0]  = 0.0007  # 'Cryodraco antarcticus'
LWa[1]  = 0.01863  #'Neopagetopsis ionah'
LWa[2]  = 0.0042  # 'Trematomus eulepidotus'
LWa[3]  = 0.0021  # 'Trematomus hansoni'
LWa[4]  = 0.00477  # 'Bathyraja maccaini'
LWa[5]  = 0.0006 # 'Chaenocephalus aceratus'
LWa[6]  = 0.0132  # 'Notothenia coriiceps'
LWa[7]  = 0.0019  # 'Pleuragramma antarctica'
LWa[8]  = 0.0045   # 'Dissostichus mawsoni'
LWa[9]  = 0.0135 # 'Macrourus whitsoni'
LWa[10] = 0.0027  # 'Lepidonotothen squamifrons'
#LWa[11] = 0.001     # 'Antimora rostrata'
LWa[11] = 0.0093 # 'Notothenia rossii'
LWa[12] = 0.00437 # 'Muraenolepis microps'
LWa[13] = 0.0005  # 'Chaenodraco wilsoni'
LWa[14] = 0.0012 # 'Chionobathyscus dewitti'

# LWb extracted 21.07.2022 from FishBase
LWb     = [None] * len(species_names) # Initialize
LWb[0]  = 3.51  # 'Cryodraco antarcticus'
LWb[1]  = 2.762  #'Neopagetopsis ionah'
LWb[2]  = 3.32  # 'Trematomus eulepidotus'
LWb[3]  = 3.52  # 'Trematomus hansoni'
LWb[4]  = 3.162  # 'Bathyraja maccaini'
LWb[5]  = 3.63  # 'Chaenocephalus aceratus'
LWb[6]  = 3.09  # 'Notothenia coriiceps'
LWb[7]  = 3.41  # 'Pleuragramma antarctica'
LWb[8]  = 3.24   # 'Dissostichus mawsoni'
LWb[9]  = 3.15 # 'Macrourus whitsoni'
LWb[10] = 3.41  # 'Lepidonotothen squamifrons'
#LWb[11] = 3.52   # 'Antimora rostrata'
LWb[11] = 3.07 # 'Notothenia rossii'
LWb[12] = 3.11 # 'Muraenolepis microps'
LWb[13] = 3.79  # 'Chaenodraco wilsoni'
LWb[14] = 3.5 # 'Chionobathyscus dewitti'

# Linf extracted 21.07.2022 from FishBase
# * Linf estimated from Lmax using Froese and Binohlan (2000) Eq. (5).
Linf     = [None] * len(species_names) # Initialize
Linf[0]  = 50.66374052  # 'Cryodraco antarcticus'*
Linf[1]  = 58.12886422  #'Neopagetopsis ionah'*
Linf[2]  = 26.5  # 'Trematomus eulepidotus'
Linf[3]  = 36.5  # 'Trematomus hansoni'
Linf[4]  = 123.061517  # 'Bathyraja maccaini'*
Linf[5]  = 70.4  # 'Chaenocephalus aceratus'
Linf[6]  = 62  # 'Notothenia coriiceps'
Linf[7]  = 25.1  # 'Pleuragramma antarctica'
Linf[8]  = 183   # 'Dissostichus mawsoni'
Linf[9]  = 92 # 'Macrourus whitsoni'
Linf[10] = 56.7  # 'Lepidonotothen squamifrons'
#Linf[11] = 66   # 'Antimora rostrata'
Linf[11] = 87 # 'Notothenia rossii'
Linf[12] = 36.60305736 # 'Muraenolepis microps'*
Linf[13] = 44.82252387  # 'Chaenodraco wilsoni'*
Linf[14] = 62.21264207 # 'Chionobathyscus dewitti'*


In [5]:
#----
# get indices of depth levels of species
#----

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

file_mesh = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/files_toothfish_project_AGI/Mesh_ancillary_information_v20220919.nc'
data_levels = xr.open_dataset(file_mesh)
levels      = data_levels['depth'].values # 88 levels
data_levels.close()

index_min_depth     = [None] * len(species_names) # Initialize
index_max_depth     = [None] * len(species_names) # Initialize

for i,name in enumerate(species_names):
    # Note Cara: order of entries in depth_min etc. is not alphabetical!!!!
    # here, I have adapted the index from "i" to "iii" to read in the correct depth ranges
    # for the current species
    iii = species_list.index(species_names[i]) # get index of current species as contained in depth_min etc.
    index_min_depth[i] = find_nearest(levels,depth_min[iii])
    index_max_depth[i] = find_nearest(levels,depth_max[iii])
    # now, index_min_depth should contain the correct values according to alphabetical order as looped over
    # NOTE: given how python is indexing, I think I then need to index over index_min_depth:index_max_depth+1
    # (was index_min_depth:index_max_depth before)
    

In [7]:
#-----
# as a test:
# calculate AGI for monthly clim
# is 10% of habitat below AGI_crit?
#-----

# constants
d = 0.7 # n.d.; metabolic scaling coefficient
j1 = 4500 # K; "Anabolism activation energy divided by Boltzmann constant" (Clarke2021)
j2 = 8000 # K; "Catabolism activation energy divided by Boltzmann constant" (Clarke2021)
j_diff = j2-j1
# from Clarke2021: "Somatic (or biomass) growth can be expressed as the difference between anabolism and catabolism"

read_Annes_values = False
if read_Annes_values:
    # Init
    AGI_crit = [None] * no_species # empty list
    o2thresh = [None] * no_species # empty list
    TPref    = [None] * no_species # empty list

    # read in AGI_crit that Anne has calculated
    #files = Path(basepath + "thresholds/").glob('AGIcrit*') 
    #for ifile,file in enumerate(files):   
    for ii in range(0,len(species_names)):
        file = 'AGIcrit_'+species_names_[ii]+'.nc'
        #print(file)
        ff     = xr.open_dataset(basepath+'thresholds/'+file)
        try:
            AGI_crit[ii]        = ff['AGIcrit'].values[0][0]
        except: # for some, the variable name is wrong (values seem ok)
            print(file)
            AGI_crit[ii]        = ff['o2thresh'].values[0][0] # in some files, variable is wrongly labeled
        ff.close()

    # read in o2thresh that Anne has calculated
    #files = Path(basepath + "thresholds/").glob('o2thresh*') 
    #for ifile,file in enumerate(files):  
    for ii in range(0,len(species_names)):
        file = 'o2thresh_'+species_names_[ii]+'.nc'
        ff     = xr.open_dataset(basepath+'thresholds/'+file)
        o2thresh[ii]        = ff['o2thresh'].values[0][0]
        ff.close()

    # read in TPref that Anne has calculated
    #files = Path(basepath + "thresholds/").glob('Tpref*') 
    #for ifile,file in enumerate(files):   
    #    ff     = xr.open_dataset(file)
    for ii in range(0,len(species_names)):
        file = 'Tpref_'+species_names_[ii]+'.nc'
        ff     = xr.open_dataset(basepath+'thresholds/'+file)
        TPref[ii]        = ff['Tpref'].values[0][0]
        ff.close()

#-----
# NOTE that order of species in LWa etc is not the same as in "species_names"
# account for that further down!!!!
#-----
# UPDATE Jan 2023: the info below is not needed anymore with updated calculation of AGI
# calculate Winf and W for each species
#Winf = np.asarray(LWa)*(np.asarray(Linf)**np.asarray(LWb))
#W    = (1./3.)*Winf
#print('Winf:',Winf)
if read_Annes_values:
    print('AGI_crit:',AGI_crit)
    print('o2thresh:',o2thresh)
    print('TPref:',TPref)

#--------
# NOTE: I don't think the order provided in LWa,LWb & Linf matches the order in which AGIcrit etc are loaded here!
# careful when indexing
# -> see above for finding the indices for the depth range!
#--------




In [8]:
for ii in range(0,len(species_names)):
    file = 'Tpref_'+species_names_[ii]+'.nc'  
    print(file)
    
print(species_names)
    
#species_list = ['Cryodraco antarcticus','Neopagetopsis ionah','Trematomus eulepidotus',\
#               'Trematomus hansoni','Bathyraja maccaini','Chaenocephalus aceratus',\
#               'Notothenia coriiceps','Pleuragramma antarctica','Dissostichus mawsoni',\
#               'Lepidonotothen squamifrons','Notothenia rossii','Muraenolepis microps',\
#               'Chaenodraco wilsoni','Chionobathyscus dewitti']


Tpref_Galiteuthis_glacialis.nc
Tpref_Mesonychoteuthis_hamiltoni.nc
Tpref_Notothenia_rossii.nc
Tpref_Pleuragramma_antarctica.nc
Tpref_Trematomus_hansoni.nc
Tpref_Chaenodraco_wilsoni.nc
Tpref_Cryodraco_antarcticus.nc
Tpref_Kondakovia_longimana.nc
Tpref_Trematomus_eulepidotus.nc
Tpref_Macrourus_whitsoni.nc
Tpref_Notothenia_coriiceps.nc
Tpref_Neopagetopsis_ionah.nc
Tpref_Lepidonotothen_squamifrons.nc
Tpref_Chaenocephalus_aceratus.nc
Tpref_Chionobathyscus_dewitti.nc
Tpref_Muraenolepis_microps.nc
Tpref_Bathyraja_maccaini.nc
Tpref_Dissostichus_mawsoni.nc
['Galiteuthis glacialis', 'Mesonychoteuthis hamiltoni', 'Notothenia rossii', 'Pleuragramma antarctica', 'Trematomus hansoni', 'Chaenodraco wilsoni', 'Cryodraco antarcticus', 'Kondakovia longimana', 'Trematomus eulepidotus', 'Macrourus whitsoni', 'Notothenia coriiceps', 'Neopagetopsis ionah', 'Lepidonotothen squamifrons', 'Chaenocephalus aceratus', 'Chionobathyscus dewitti', 'Muraenolepis microps', 'Bathyraja maccaini', 'Dissostichus mawsoni']

In [9]:
#-----
# calculate AGI
# part I: load pO2 & t_insitu monthly climatologies
#-----

path1 = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/files_toothfish_project_AGI/simAssp585_monthly/'

# pO2 data (mbar)
file_pO2   = path1+'/pO2_fesom_simA_monthly_clim_1995_2014_v2.nc'
data_pO2   = xr.open_dataset(file_pO2)
pO2        = data_pO2['pO2'].values
data_pO2.close()

# T insitu (deg C)
file_t_insitu   = path1+'/t_insitu_fesom_simA_monthly_clim_1995_2014_v2.nc'
data_t_insitu   = xr.open_dataset(file_t_insitu)
t_insitu        = data_t_insitu['t_insitu'].values
data_t_insitu.close()

# volume data 
data_vol   = xr.open_dataset(file_mesh)
vol        = data_vol['volume']
data_vol.close()

print(vol.shape)
print(t_insitu.shape)
print(pO2.shape)


(88, 360, 1440)
(12, 88, 360, 1440)
(12, 88, 360, 1440)


In [10]:
#------
# weighted quantile function
# from here: https://stackoverflow.com/questions/21844024/weighted-percentile-using-numpy
#------
########
# to be double-check carefully! 
########

def weighted_quantile(values, quantiles, sample_weight=None, 
                      values_sorted=False, old_style=False):
    """ Very close to numpy.percentile, but supports weights.
    NOTE: quantiles should be in [0, 1]!
    :param values: numpy.array with data
    :param quantiles: array-like with many quantiles needed
    :param sample_weight: array-like of the same length as `array`
    :param values_sorted: bool, if True, then will avoid sorting of
        initial array
    :param old_style: if True, will correct output to be consistent
        with numpy.percentile.
    :return: numpy.array with computed quantiles.
    """
    values = np.array(values)
    quantiles = np.array(quantiles)
    if sample_weight is None:
        sample_weight = np.ones(len(values))
    sample_weight = np.array(sample_weight)
    assert np.all(quantiles >= 0) and np.all(quantiles <= 1), \
        'quantiles should be in [0, 1]'

    if not values_sorted:
        sorter = np.argsort(values)
        values = values[sorter]
        sample_weight = sample_weight[sorter]

    weighted_quantiles = np.cumsum(sample_weight) - 0.5 * sample_weight
    if old_style:
        # To be convenient with numpy.percentile
        weighted_quantiles -= weighted_quantiles[0]
        weighted_quantiles /= weighted_quantiles[-1]
    else:
        weighted_quantiles /= np.sum(sample_weight)
    return np.interp(quantiles, weighted_quantiles, values)

In [23]:
#-----
# part II: calculate Tpref, O2thresh, critical AGI
# save as netcdf
#-----

#for i,name in enumerate(species_names):
for i in range(7,8):#len(species_names)):
    print(i,species_names[i])
    print(habitat_file_names[i])
    
    ### Get the in-habitat data for each species over the relevant depth range ###
    file_habitat = habitat_file_names[i]
    data_hab     = xr.open_dataset(file_habitat) # 2D habitat presence/absence data (1 for present or 0 for absent)
    # make sure the habitat field has dimensions depth x lat x lon before applying it to the respective fields
    num_depth = len(np.arange(index_min_depth[i],index_max_depth[i]+1,1)) 
    hab_field = np.tile(data_hab['presence'].values,(num_depth,1,1))
    data_hab.close()
    
    ### select the data/volume over the relevant depth range, correct with habitat array
    # NOTE: index_min_depth[i]:index_max_depth[i]    or.    index_min_depth[i]:index_max_depth[i]+1   ?
    vol_inhabitat = (vol.values[index_min_depth[i]:index_max_depth[i]+1,:,:]*hab_field) 
    dataout1      = (pO2[:,index_min_depth[i]:index_max_depth[i]+1,:,:]*hab_field)
    dataout2      = (t_insitu[:,index_min_depth[i]:index_max_depth[i]+1,:,:]*hab_field)
    #print(dataout1.shape) # month x depth x lat x lon

    # use all months
    dataout1b = dataout1[:,:,:,:].ravel() # pO2
    dataout2b = dataout2[:,:,:,:].ravel() # temp
    # make sure volume array has dimensions months x depths x lat x lon before applying ravel()
    vol_inhabitat_b = np.tile(vol_inhabitat,(12,1,1,1)).ravel() 
    
    # make sure to only keep nodes that are available for current species
    ind = np.where((~np.isnan(dataout1b)))[0] # every available node
    dataout1b = dataout1b[ind]
    dataout2b = dataout2b[ind]
    vol_inhabitat_b = vol_inhabitat_b[ind]
    ind = np.where((dataout1b!=0))[0] # every available node
    dataout1b = dataout1b[ind] # pO2
    dataout2b = dataout2b[ind] # temp
    vol_inhabitat_b = vol_inhabitat_b[ind]
    del ind
    
  #  print('Min/Max pO2 in habitat:',np.min(dataout1b),np.max(dataout1b))
  #  print('Min/Max t insitu in habitat:',np.min(dataout2b),np.max(dataout2b))
    
    totalvol_inhabitat = np.nansum(vol_inhabitat_b) # total in-habitat volume (accounting for bathymetry)
  #  print('totalvol_inhabitat (3D):',totalvol_inhabitat) 
    vol_weights = vol_inhabitat_b/np.sum(vol_inhabitat_b)
    #print(np.sum(vol_weights)) # as a check, should be 1
    
    # order of species in W, Winf etc not the same as in this i-loop!
    # correct for that here (same was done for index_min_depth and index_max_depth)
    iii = species_list.index(species_names[i])
    
    #------
    # get Tpref & O2thresh
    #   Tpref: 50th percentile in habitat (volume weighted)
    #   O2thresh: 10th percentile in habitat (volume weighted)
    tpref_species    = weighted_quantile(dataout2b, [0.5], sample_weight=vol_weights)
    o2thresh_species = weighted_quantile(dataout1b, [0.1], sample_weight=vol_weights)    
    # from Anne's files: TPref[i],o2thresh[i]
    if read_Annes_values:
        print('Compare TPref from file and calculated:',TPref[i],tpref_species)
        print('Compare O2thresh from file and calculated:',o2thresh[i],o2thresh_species)
    else:
        print('TPref:',tpref_species)
        print('O2thresh:',o2thresh_species)
    #------
    
    #----
    # calculate AGIcrit: 10th percentile in habitat (volume wieghted)
    #
    # version used before Jan 2023
   # w_ratio_old = (W[iii]**(1-d))/(Winf[iii]**(1-d))
    #a1 = np.exp((j_diff/(tpref_species+273.15)) - (j_diff/(dataout2b+273.15)))
    #
    #AGI = dataout1b/(o2thresh_species*w_ratio*a1)
    ##print('Min/Max AGI in 3D habitat:',np.min(AGI),np.max(AGI))
    
    
    # UPDATE Jan 2023: adapt calculation of AGI to what is in Anne's paper (W, W_inf etc not needed anymore)
    w_ratio = ((1./3.)**(1-d))
    a1 = np.exp((j_diff/(tpref_species+273.15)) - (j_diff/(dataout2b+273.15)))
    if read_Annes_values:
        print('Compare w_ratio between old and new formulation:',w_ratio_old,w_ratio)
    else:
        print('w_ratio:',w_ratio)
    AGI = dataout1b/(o2thresh_species*w_ratio*a1)
    
    
    res = weighted_quantile(AGI, [0.1], sample_weight=vol_weights)
    print('AGI_crit calculated:',res[0])
    if read_Annes_values:
        print('AGI_crit from nc file:',AGI_crit[i])    
    
    ind_hab = np.where(AGI>res[0])[0]
    vol_hab = np.sum(vol_inhabitat_b[ind_hab])
    #----
    
    #----
    # save thresholds as netcdf
    #----
    save_as_netcdf = True
    if save_as_netcdf:
        savepath = '/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/AGI_toothfish_project/new_thresholds/'
        
        #---
        # habitable volume, present-day
        #---
        netcdf_name = 'volume_habitat_'+species_names[i].replace(' ','_')+'_based_on_monthly_clim_1995_2014.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.info = "Volume in habitat above AGIcrit" 
            w_nc_fid.note = "calculation based on monthly data, divided by 12 here to get annual mean" 
            w_nc_fid.author = "Cara Nisen & Anne Moree, Nov 2022" 
            w_nc_fid.units = "m3" ;
            w_nc_fid.source_pO2    = file_pO2
            w_nc_fid.source_Tpref    = file_t_insitu
            w_nc_fid.script    = '/home/ollie/ncara/scripts/plot_AGI_toothfish_TPref_O2thresh_save_as_netcdf.ipynb'
            w_nc_fid.depth_min = depth_min[iii]
            w_nc_fid.depth_max = depth_max[iii]
            try:
                w_nc_fid.LWa  = LWa[iii]
                w_nc_fid.LWb  = LWb[iii]
                w_nc_fid.Linf = Linf[iii]
            except: # the above info is not needed anymore with updated AGI calculation
                pass
            # create dimension & variable
            w_nc_fid.createDimension('num', 1)  
            w_nc_fid.createVariable('volume_habitat', 'f4',('num'))
            # write variable
            w_nc_fid.variables['volume_habitat'][:] = vol_hab/12
            w_nc_fid.close()
            
        #---
        # Tpref
        #---
        netcdf_name = 'Tpref_'+species_names[i].replace(' ','_')+'_based_on_monthly_clim_1995_2014.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.info = "Tpref calculated as 50th percentile from lonxlatxdepth insitu T data in-habitat, volume weighted." ;
            w_nc_fid.author = "Cara Nisen & Anne Moree, Nov 2022" ;
            w_nc_fid.reference = "Tpref as described in Clarke et al. (2021)" ;
            w_nc_fid.units = "C" ;
            w_nc_fid.method = "50th percentile of the field using Python, weighted by volume" ;
            w_nc_fid.source_pO2    = file_pO2
            w_nc_fid.source_Tpref    = file_t_insitu
            w_nc_fid.script    = '/home/ollie/ncara/scripts/plot_AGI_toothfish_TPref_O2thresh_save_as_netcdf.ipynb'
            w_nc_fid.depth_min = depth_min[iii]
            w_nc_fid.depth_max = depth_max[iii]
            try:
                w_nc_fid.LWa  = LWa[iii]
                w_nc_fid.LWb  = LWb[iii]
                w_nc_fid.Linf = Linf[iii]
            except: # the above info is not needed anymore with updated AGI calculation
                pass
            # create dimension & variable
            w_nc_fid.createDimension('num', 1)  
            w_nc_fid.createVariable('Tpref', 'f4',('num'))
            # write variable
            w_nc_fid.variables['Tpref'][:] = tpref_species
            w_nc_fid.close()
            
        #---
        # O2threshold
        #---
        netcdf_name = 'o2thresh_'+species_names[i].replace(' ','_')+'_based_on_monthly_clim_1995_2014.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.info = "o2thresh calculated as 10th percentile from lonxlatxdepth pO2 data in-habitat, volume weighted." ;
            w_nc_fid.author = "Cara Nisen & Anne Moree, Nov 2022" ;
            w_nc_fid.reference = "o2thresh as described in Clarke et al. (2021)" ;
            w_nc_fid.units = "C" ;
            w_nc_fid.method = "10th percentile of the field using Python, weighted by volume" ;
            w_nc_fid.source_pO2    = file_pO2
            w_nc_fid.source_Tpref    = file_t_insitu
            w_nc_fid.script    = '/home/ollie/ncara/scripts/plot_AGI_toothfish_TPref_O2thresh_save_as_netcdf.ipynb'
            w_nc_fid.depth_min = depth_min[iii]
            w_nc_fid.depth_max = depth_max[iii]
            try:
                w_nc_fid.LWa  = LWa[iii]
                w_nc_fid.LWb  = LWb[iii]
                w_nc_fid.Linf = Linf[iii]
            except: # the above info is not needed anymore with updated AGI calculation
                pass
            # create dimension & variable
            w_nc_fid.createDimension('num', 1)  
            w_nc_fid.createVariable('o2thresh', 'f4',('num'))
            # write variable
            w_nc_fid.variables['o2thresh'][:] = o2thresh_species
            w_nc_fid.close()
            
        #---
        # AGIcrit
        #---
        netcdf_name = 'AGIcrit_'+species_names[i].replace(' ','_')+'_based_on_monthly_clim_1995_2014.nc'
        if not os.path.exists(savepath+netcdf_name):
            print('Create file '+savepath+netcdf_name)
            w_nc_fid = Dataset(savepath+netcdf_name, 'w', format='NETCDF4_CLASSIC')
            w_nc_fid.info = "AGIcrit calculated as 10th percentile from lonxlatxdepth insitu T data & pO2 in-habitat, volume weighted." ;
            w_nc_fid.author = "Cara Nisen & Anne Moree, Nov 2022" ;
            w_nc_fid.reference = "AGIcrit as described in Clarke et al. (2021)" ;
            w_nc_fid.units = "C" ;
            w_nc_fid.method = "10th percentile of the field using Python, weighted by volume" ;
            w_nc_fid.source_pO2    = file_pO2
            w_nc_fid.source_Tpref    = file_t_insitu
            w_nc_fid.script    = '/home/ollie/ncara/scripts/plot_AGI_toothfish_TPref_O2thresh_save_as_netcdf.ipynb'
            w_nc_fid.depth_min = depth_min[iii]
            w_nc_fid.depth_max = depth_max[iii]
            try:
                w_nc_fid.LWa  = LWa[iii]
                w_nc_fid.LWb  = LWb[iii]
                w_nc_fid.Linf = Linf[iii]
            except: # the above info is not needed anymore with updated AGI calculation
                pass
            # create dimension & variable
            w_nc_fid.createDimension('num', 1)  
            w_nc_fid.createVariable('AGIcrit', 'f4',('num'))
            # write variable
            w_nc_fid.variables['AGIcrit'][:] = res[0]
            w_nc_fid.close()
        print('Saved thresholds for',species_names[i])
            
    del a1,w_ratio,totalvol_inhabitat,vol_inhabitat_b,vol_inhabitat
    del dataout1b,dataout2b,AGI
    del dataout1,dataout2
    print ('')
    
print('done')


7 Kondakovia longimana
/work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/AGI_toothfish_project/updated_habitat_files/Kondakovia_longimana_modelgrid_boolean.nc
TPref: [0.64053279]
O2thresh: [121.53898621]
w_ratio: 0.7192230933248643
AGI_crit calculated: 1.2658287271283182
Create file /work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/AGI_toothfish_project/new_thresholds/volume_habitat_Kondakovia_longimana_based_on_monthly_clim_1995_2014.nc
Create file /work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/AGI_toothfish_project/new_thresholds/Tpref_Kondakovia_longimana_based_on_monthly_clim_1995_2014.nc
Create file /work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/AGI_toothfish_project/new_thresholds/o2thresh_Kondakovia_longimana_based_on_monthly_clim_1995_2014.nc
Create file /work/ollie/ncara/fesom/fesom-1.4-recom/HLRN/AGI_toothfish_project/new_thresholds/AGIcrit_Kondakovia_longimana_based_on_monthly_clim_1995_2014.nc
Saved thresholds for Kondakovia longimana

done
